## KDE Visualization Notebook
____
This notebook aims to provide an animation of CMS or LHCb KDEs calculated from tracks. In order to run this notebook, you need a .txt file corresponding to the grid of x,y values for every z bin. This .txt file is automatically generated for the last event in a ROOT file generated by the ./make_histogram_from_tracks code in ana/build, writing the file to the build folder (move to use) (probably needs to be edited to create a file for anything except the last event in the file...). Be sure to edit the grid size parameters in order to run this notebook. I have included two files in the notebooks directory so that anyone can run the notebook.

In [3]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import os
import time
%matplotlib inline

In [6]:
file = 'grid-lhcb2.txt'
wf = open(file, 'r')
lines = wf.readlines()

In [9]:
# This cell reads the text file into a 3-dimensional numpy array.
grid_reconstructed = np.empty((4000,9,9))

zcount = 0
xcount = 0

for i in range(len(lines)):
    
    line = lines[i]
    
    if len(line) == 1:
        xcount = 0
        zcount += 1
        continue
        
    vals = [float(line.split()[i]) for i in range(len(line.split()))]
    
    grid_reconstructed[zcount, xcount, :] = vals
    
    if zcount%1000 == 0 and xcount==0:
        print(np.round(100*zcount/grid_reconstructed.shape[0]), '% Complete')
        
    xcount += 1
    

0.0 % Complete
25.0 % Complete
50.0 % Complete


In [10]:
f = uproot.open('kernel_HLT1CPU_D0piMagUp_10.root')
kernel= f["kernel"]
zdata = kernel["zdata"].array()
xmax = kernel["xmax"].array()
ymax = kernel["ymax"].array()

In [11]:
#Savitzky-Golay filter used to set maximum of colorbar to mitigate "jumping effect" in animation
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    window_size = np.abs(np.int(window_size))
    order = np.abs(int(order))
#     except ValueError, msg:
#         raise ValueError("window_size and order have to be of type int")
    if not window_size % 2 == 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [ ]:
font = {'family': 'serif',
        'color':  'red',
        'weight': 'bold',
        'size': 16}

zrange = np.linspace(-100,300,4000)#np.linspace(-25,25,10000)
xrange = np.linspace(-0.40,0.41,9)#np.linspace(-0.40,0.41,9)
yrange = np.linspace(-0.40,0.41,9)#np.linspace(-0.40,0.41,9)
xWidth = np.abs(xrange[1] - xrange[0])
yWidth = np.abs(yrange[1] - yrange[0])

xGrid, yGrid = np.meshgrid(xrange, yrange)

savgol = savitzky_golay(np.array(zdata[-1].tolist()), 51, 3)

for i in range(len(grid_reconstructed[:,0,0]))[:2200]:
    
    if not i%5==0:
        continue
    
    fig = plt.figure(figsize=(16,12), linewidth=10, edgecolor='k')
    fig.set_facecolor('white')
    ax1 = fig.add_subplot(2, 2, (1,2))
    ax2 = fig.add_subplot(2, 2, 3, projection='3d')
    ax3 = fig.add_subplot(2, 2, 4)
    
    text = 'z = ' + str(np.round(zrange[i],2)) + ' (' + str(i) + ')'
    fig.text(0.8,0.89,text,fontdict=font)

    ax1.plot(zrange,zdata[-1])
    ax1.vlines(zrange[i],ymin = 0,ymax = 2500,color = 'r')
    ax1.set_title('KDE')
    ax1.set_xlabel('z')

    ax2.bar3d(xGrid.ravel(), 
              yGrid.ravel(),
              np.zeros(grid_reconstructed[i,:,:].ravel().shape), 
              xWidth, 
              yWidth, 
              grid_reconstructed[i,:,:].ravel(),
              shade = True, 
              color = 'lightskyblue')
    ax2.set_title("Bar Graph")
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_zlabel('Track Density')
    ax2.set_zlim([0,savgol[i]])

    xSpace2 = np.arange(-0.40,0.41+xWidth,xWidth)
    ySpace2 = np.arange(-0.40,0.41+yWidth,yWidth)
    xGrid2, yGrid2 = np.meshgrid(xSpace2, ySpace2)

    maxindflat = np.argmax(grid_reconstructed[i,:,:])
    maxindtuple = np.unravel_index(maxindflat,grid_reconstructed[i,:,:].shape)
    
    c = ax3.pcolormesh(xGrid2, yGrid2, grid_reconstructed[i,:,:], 
                       cmap='Blues', 
                       shading = 'auto',
                       vmin = 0, 
                       vmax = savgol[i])
    ax3.plot(xGrid[maxindtuple],yGrid[maxindtuple],'o',color='red')
    fig.colorbar(c, ax=ax3, label = 'Track Density')
    ax3.set_title('Color Map')
    ax3.set_xlabel('x')
    ax3.set_ylabel('y')
    
    # uncomment below to save figures (can download and create gif, have not figured out how to do animation in jupyter notebook)
#     s = f'{i:05d}'
#     fname = '/home/ekauffma/cmsanimation/lhcbaniPRES_' + s + '.png'
#     fig1 = plt.gcf()
#     plt.show()
#     fig1.savefig(fname)
    

<ipython-input-11-80fdf7d03e60>:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  window_size = np.abs(np.int(window_size))
<ipython-input-12-542f3322c00c>:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(16,12), linewidth=10, edgecolor='k')


KeyboardInterrupt: 